# "Reuters Dataset 1 Metamodel Long-Short BackTest"
> "Reuters Dataset 1 Metamodel Long-Short BackTest"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

Meta-model Analysis - Model Stacking

Meta-model analysis was carried out to see if the different models predict better on the differeent feature spaces of the test data set. Model outputs were trained as inputs into gradient boosting models (Catboost, LGBM and ExtraTreesClassifiers).

The CatBoost meta-model accuracy (56%) improved on the best accuarcy score (55%) and went on to outperform the SentiMetre Model 2 ranked by return.

In [64]:
#hide_input
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [65]:
#hide_input
df_tes=pd.read_csv('D:\\Downloads\\Metamodelsample1.csv')
df_tes.reset_index(drop=True, inplace=True)
df_tes.sort_values(by=['Date'], ascending=True,inplace=True)
df_tes['label']=0
df_tes['label']=np.where(df_tes['excess_return']>0,1,np.where(df_tes['excess_return']<0,-1,df_tes['label']))

In [66]:
#Accuracy
#hide_input
list_accuracy=['tr_pred','catboost_pred','lgbm_pred','xtrees_pred']
for acc in list_accuracy:
  print('Accuracy',acc,len(df_tes.loc[df_tes.label==df_tes[acc].astype(float)])/len(df_tes))


Accuracy tr_pred 0.547798066595059
Accuracy catboost_pred 0.5596133190118152
Accuracy lgbm_pred 0.5528105979233798
Accuracy xtrees_pred 0.573218761188686


In [67]:

#hide_input

#hide
df_tes['returnpred']=df_tes['1_day_return']*df_tes['tr_pred']
df_tes['returnpredcat']=df_tes['1_day_return']*df_tes['catboost_pred']
df_tes['returnpredlgbm']=df_tes['1_day_return']*df_tes['lgbm_pred']
df_tes['returnpredxtree']=df_tes['1_day_return']*df_tes['xtrees_pred']


# In[266]:


#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190201].loc[df_tes.Date<20190301].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_long=df_tes.loc[df_tes.pred==1].copy().reset_index(drop=True)
df_short=df_tes.loc[df_tes.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)


# In[267]:
#hide
df_tes['dailyaveragereturn']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturn']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['returnpred']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))


#hide
df_tes['dailyaveragereturncat']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturncat']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['returnpredcat']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

#hide
df_tes['dailyaveragereturnlgbm']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturnlgbm']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['returnpredlgbm']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_tes['dailyaveragereturnxtree']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturnxtree']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['returnpredxtree']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))



# In[268]:


#hide
#df_tes['2dayreturn']=0.0
#df_tes['Daytwo']=df_tes["Date"].shift(1)
#for date in df_tes['Daytwo'].unique():
    #if date>0:
        #df_tes.loc[df_tes['Daytwo']==date,'2dayreturn']=np.prod(1+(df_tes.loc[df_tes['Daytwo']==date]['2_day_return']))**(1/len(df_tes.loc[df_tes['Daytwo']==date]))
        #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))


# In[269]:


#hide
df_a=df_tes.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)


# In[270]:


#hide
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])

df_a['cumreturn1c']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1c']=np.prod(df_a['dailyaveragereturncat'][:l+1])

    
df_a['cumreturn1d']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1d']=np.prod(df_a['dailyaveragereturnlgbm'][:l+1])

df_a['cumreturn1e']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1e']=np.prod(df_a['dailyaveragereturnxtree'][:l+1])
# **Long-Short Portfolio**

# In[271]:


#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(int)
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPCa.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJIa.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')


# In[272]:


#hide_input
df_a[['Date','cumreturn1b','cumreturndow','cumreturnsp500']]


# In[273]:


#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='Long-Short Portfolio Backtest: Sentimetre Model 2 (blue) vs Vader (brown) vs Linear Classifier (orange) vs Dow (black) vs SP500 (red) '
)
a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_line(color='brown').encode(
   x='Date:T',
   y='cumreturn1d:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_a).mark_line(color='orange').encode(
    x='Date:T',
    y='cumreturn1e:Q',

).properties(
    width=1000,
    height=1000
)

a6 = alt.Chart(df_a).mark_line(color='pink').encode(
    x='Date:T',
    y='cumreturn1c:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5 +a6

alt.LayerChart(...)

In [ ]:
#hide_input

# **Long portfolio**

# In[274]:


#hide
df_long['dailyaveragereturn']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturn']=np.prod(1+(df_long.loc[df_long['Date']==date]['returnpred']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

#hide
df_long['dailyaveragereturncat']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturncat']=np.prod(1+(df_long.loc[df_long['Date']==date]['returnpredcat']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

    
df_long['dailyaveragereturnlgbm']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturnlgbm']=np.prod(1+(df_long.loc[df_long['Date']==date]['returnpredlgbm']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))


df_long['dailyaveragereturnxtree']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturnxtree']=np.prod(1+(df_long.loc[df_long['Date']==date]['returnpredxtree']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))
df_a=df_long.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])

df_a['cumreturn1c']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1c']=np.prod(df_a['dailyaveragereturncat'][:l+1])

    
df_a['cumreturn1d']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1d']=np.prod(df_a['dailyaveragereturnlgbm'][:l+1])

df_a['cumreturn1e']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1e']=np.prod(df_a['dailyaveragereturnxtree'][:l+1])

# In[275]:


#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(int)
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPCa.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJIa.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')


# In[276]:


#hide_input
df_a[['Date','cumreturn1b','cumreturndow','cumreturnsp500']]


# In[277]:


#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='Long Portfolio Backtest: Sentimetre Model 2 (blue) vs CatBoost Metamodel (pink) vs LGBM Metamodel (brown) vs Xtrees Metamodel (orange) vs  Dow (black) vs SP500 (red) '
)


a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1d:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_a).mark_line(color='orange').encode(
   x='Date:T',
    y='cumreturn1e:Q',

).properties(
    width=1000,
    height=1000
)

a6 = alt.Chart(df_a).mark_line(color='pink').encode(
    x='Date:T',
    y='cumreturn1c:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5 +a6


In [ ]:
#hide_input


# **Short Portfolio**

# In[278]:
#hide
df_short['dailyaveragereturn']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturn']=np.prod(1+(df_short.loc[df_short['Date']==date]['returnpred']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))


#hide
df_short['dailyaveragereturncat']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturncat']=np.prod(1+(df_short.loc[df_short['Date']==date]['returnpredcat']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_short['dailyaveragereturnlgbm']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturnlgbm']=np.prod(1+(df_short.loc[df_short['Date']==date]['returnpredlgbm']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_short['dailyaveragereturnxtree']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturnxtree']=np.prod(1+(df_short.loc[df_short['Date']==date]['returnpredxtree']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_short.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])


df_a['cumreturn1c']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1c']=np.prod(df_a['dailyaveragereturncat'][:l+1])
    
df_a['cumreturn1d']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1d']=np.prod(df_a['dailyaveragereturnlgbm'][:l+1])

df_a['cumreturn1e']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1e']=np.prod(df_a['dailyaveragereturnxtree'][:l+1])

# In[279]:


#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(int)
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPCa.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJIa.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')


# In[280]:


#hide_input
#df_a[['Date','cumreturn1b','cumreturndow','cumreturnsp500']]


# In[281]:


#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='Short Portfolio Backtest: Sentimetre Model 2 (blue) vs CatBoost Metamodel (pink) vs LGBM Metamodel (brown) vs Xtrees Metamodel (orange) vs  Dow (black) vs SP500 (red)  '
)


a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1d:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_a).mark_line(color='orange').encode(
    x='Date:T',
    y='cumreturn1e:Q',

).properties(
    width=1000,
    height=1000
)

a6 = alt.Chart(df_a).mark_line(color='Pink').encode(
    x='Date:T',
    y='cumreturn1c:Q',

).properties(
    width=1000,
    height=1000
)


alt.layer(a1 ,a2 ,a3 ,a4 ,a5,a6).resolve_scale()
#a1 + a2 + a3 + a4 + a5 +a6

